In [ ]:
!pip install kaggle --quiet

import os
os.environ['KAGGLE_CONFIG_DIR'] = "~/.kaggle"  # Update this path if needed

# I had to do this because it wasn't seeing my kaggle.json
os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''

!kaggle competitions download -c rsna-pneumonia-detection-challenge

import zipfile
with zipfile.ZipFile('rsna-pneumonia-detection-challenge.zip', 'r') as zip_ref:
    zip_ref.extractall('rsna_data')

import os
os.listdir('rsna_data')

In [1]:
# !pip install pydicom
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
import os
import numpy as np
import pandas as pd
import pydicom
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
import cv2

data_dir = './rsna_data/stage_2_train_images'
labels_df = pd.read_csv('./rsna_data/stage_2_train_labels.csv')

def load_dicom_image(image_path):
    dicom_data = pydicom.dcmread(image_path)

    image = dicom_data.pixel_array

    if len(image.shape) == 2:
        image = np.stack([image] * 3, axis=-1)  # Convert grayscale to RGB

    image = cv2.resize(image, (224, 224))

    image = image / 255.0

    image = (image * 255).astype(np.uint8)

    return image

images = []
labels = []
for idx, row in labels_df.iterrows():
    image_path = os.path.join(data_dir, f'{row["patientId"]}.dcm')
    images.append(load_dicom_image(image_path))
    labels.append(row["Target"])

images = np.array(images)
labels = np.array(labels)


In [4]:

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

'''train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)'''
train_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
validation_generator = val_datagen.flow(X_val, y_val, batch_size=32)


: 

In [ ]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import Input
inputs = Input(shape=(224, 224, 3))


x = inputs
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False
x = base_model(x, training=False)

# Custom classification head
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)


In [ ]:

from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
	metrics=['accuracy']
)

In [ ]:
from tensorflow.keras import callbacks
# --- TRAINING ---
early_stop = callbacks.EarlyStopping(patience=5, restore_best_weights=True, min_delta=0.001, monitor='accuracy', mode='max')
reduce_lr = callbacks.ReduceLROnPlateau(patience=3, factor=0.5, verbose=1)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stop]
)

In [ ]:
model.save("cnn_chest_pretune_resnet.keras")
from tensorflow.keras.models import load_model
#model = load_model("cnn_chest_pretune.keras")
base_model.trainable = True

In [ ]:
classifier_head_weights = [layer.get_weights() for layer in model.layers[-8:]]


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    #metrics=[AUC(multi_label=True)]
	metrics=['accuracy']
)

In [ ]:

for i, layer in enumerate(model.layers[-8:]):
    layer.set_weights(classifier_head_weights[i])

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stop, reduce_lr]
)